In [21]:
import pandas as pd
import numpy as np
import random
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf
import sqlite3
import pyspark


## dataset load

In [22]:
# netflix
netflix = 'https://raw.githubusercontent.com/CharlesFeng0314/8083Porj/main/rwa_data/Netflix.csv'
netflix = pd.read_csv(netflix)
netflix.shape

(15860, 8)

In [23]:
#amazon
amazon = 'https://raw.githubusercontent.com/CharlesFeng0314/8083Porj/main/rwa_data/amazon.csv'
amazon = pd.read_csv(amazon)
amazon.shape

(52390, 8)

In [24]:
#apple
apple = 'https://raw.githubusercontent.com/CharlesFeng0314/8083Porj/main/rwa_data/apple.csv'
apple = pd.read_csv(apple)
apple.shape

KeyboardInterrupt: 

24/11/23 19:41:49 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker
24/11/23 23:15:24 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 943883 ms exceeds timeout 120000 ms
24/11/23 23:15:24 WARN SparkContext: Killing executors is not supported by current scheduler.
24/11/23 23:15:27 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerM

In [ ]:
#hbo
hbo = 'https://raw.githubusercontent.com/CharlesFeng0314/8083Porj/main/rwa_data/hbo.csv'
hbo = pd.read_csv(hbo)
hbo.shape

(7967, 8)

In [ ]:
#hulu
hulu = 'https://raw.githubusercontent.com/CharlesFeng0314/8083Porj/main/rwa_data/hulu.csv'
hulu = pd.read_csv(hulu)
hulu.shape

(8342, 8)

## data pre-process

In [ ]:
imdb_df = pd.concat([netflix, amazon, apple, hbo, hulu], ignore_index=True)
imdb_df = imdb_df.drop_duplicates()
imdb_df = imdb_df.dropna(subset=['imdbNumVotes'])

In [ ]:
Z = 1.96  # Z-score for 95% confidence level
p = 0.5   # Estimated proportion (maximum variability)
e = 0.05  # Margin of error
N = imdb_df['imdbNumVotes']
imdb_df['post_count'] = np.ceil((Z**2 * p * (1 - p)) / (e**2 + (Z**2 * p * (1 - p)) / N))

In [ ]:
imdb_df = imdb_df.sort_values(by='imdbNumVotes', ascending=False)
imdb_df

,title,type,genres,releaseYear,imdbId,imdbAverageRating,imdbNumVotes,availableCountries,post_count
16014,The Shawshank Redemption,movie,Drama,1994.0,tt0111161,9.3,2957061.0,"DK, FI, JP, NL, NO, SE",385.0
71,The Shawshank Redemption,movie,Drama,1994.0,tt0111161,9.3,2957061.0,"AD, BA, BE, BG, CA, CV, CZ, ES, GQ, HR, HU, IN...",385.0
83057,The Shawshank Redemption,movie,Drama,1994.0,tt0111161,9.3,2957061.0,"AD, AR, BE, BO, BR, BZ, CL, CO, CR, DK, DO, EC...",385.0
90981,The Shawshank Redemption,movie,Drama,1994.0,tt0111161,9.3,2957061.0,JP,385.0
83007,The Dark Knight,movie,"Action, Crime, Drama",2008.0,tt0468569,9.0,2938846.0,"AD, AG, AR, BA, BB, BE, BG, BO, BR, BS, BZ, CL...",385.0
...,...,...,...,...,...,...,...,...,...
72415,Rhett and Link in France,movie,"Comedy, Family, Talk-Show",2014.0,tt5544480,6.8,5.0,US,5.0
25224,AI Rising,movie,"Documentary, News",2023.0,tt27650201,7.2,5.0,"AD, BE, CH, FR, GB, GG, IE, IT, MC, SM, US, VA",5.0
58813,Wayne,movie,"Drama, Romance, Short",2015.0,tt3712354,8.0,5.0,"GB, GG",5.0
94658,Psychic Agents,movie,"Crime, Mystery, Romance",2017.0,tt6968706,5.0,5.0,JP,5.0


## Fetch posts

In [ ]:
conf = SparkConf() \
    .setAppName("Fast") \
    .set("spark.default.parallelism", "8") \
    .set("spark.driver.memory", "4g") \
    .set("spark.executor.memory", "4g")

Fast_spark = SparkSession.builder.config(conf=conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/23 19:19:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 50941)
Traceback (most recent call last):
  File "/Users/fengcharles/anaconda3/envs/pyspark/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/fengcharles/anaconda3/envs/pyspark/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/fengcharles/anaconda3/envs/pyspark/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/fengcharles/anaconda3/envs/pyspark/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/fengcharles/anaconda3/envs/pyspark/lib/python3.9/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Users/fengcharles/anaconda3/envs/pyspark/lib/python3.9/site-packages/pys

In [ ]:
def load_json_as_dict(folder_path):
    json_dict = {}
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):  
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_json(file_path)
                json_dict[os.path.splitext(filename)[0]] = df
            except Exception as e:
                print(f"无法读取文件 {filename}: {e}")
    
    return json_dict

folder_path = "/Users/fengcharles/Desktop/8083Porj/comments data"  
comments = load_json_as_dict(folder_path)

print("读取到的JSON文件有:", comments.keys())

读取到的JSON文件有: dict_keys(['part-06', 'part-01', 'part-03', 'sample', 'part-02', 'part-05', 'part-04'])


In [ ]:
del comments['sample']

In [ ]:
spark_dfs = {}
for table_name, df in comments.items():
    try:
        # 将 pandas DataFrame 转换为 PySpark DataFrame
        spark_df = Fast_spark.createDataFrame(df)
        spark_dfs[table_name] = spark_df
        print(f"表 {table_name} 已成功转换为 PySpark DataFrame")
    except Exception as e:
        print(f"转换表 {table_name} 时出错: {e}")

# 显示 PySpark DataFrame 的部分数据
for table_name, spark_df in spark_dfs.items():
    print(f"表 {table_name} 的前 5 行:")
    spark_df.show(5)


表 part-06 已成功转换为 PySpark DataFrame
表 part-01 已成功转换为 PySpark DataFrame
表 part-03 已成功转换为 PySpark DataFrame
转换表 part-02 时出错: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.readRDDFromFile.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.api.java.JavaRDD$.readRDDFromInputStream(JavaRDD.scala:252)
	at org.apache.spark.api.java.JavaRDD$.readRDDFromFile(JavaRDD.scala:239)
	at org.apache.spark.api.python.PythonRDD$.readRDDFromFile(PythonRDD.scala:289)
	at org.apache.spark.api.python.PythonRDD.readRDDFromFile(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j

24/11/23 19:41:27 WARN TaskSetManager: Stage 0 contains a task of very large size (84443 KiB). The maximum recommended task size is 1000 KiB.
ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/Users/fengcharles/anaconda3/envs/pyspark/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/fengcharles/anaconda3/envs/pyspark/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Users/fengcharles/anaconda3/envs/pyspark/lib/python3.9/socket.py", line 716, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 